In [136]:
# import importlib

# package_name = "datasets"

# try:
#     importlib.import_module(package_name)
#     print(f"{package_name} is already installed.")
# except ImportError:
#     print(f"{package_name} is not installed. Installing it now...")
#     !pip install {package_name}
#     print(f"{package_name} has been successfully installed.")

In [137]:
# import importlib

# package_name = "accelerate"

# try:
#     importlib.import_module(package_name)
#     print(f"{package_name} is already installed.")
# except ImportError:
#     print(f"{package_name} is not installed. Installing it now...")
#     !pip install {package_name}
#     print(f"{package_name} has been successfully installed.")

In [138]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForPreTraining
from transformers import BertTokenizer, AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [139]:
# from google.colab import drive
# drive.mount("/content/drive")

# %cd "/content/drive/MyDrive/6386"

# # from colab.file_utils import load_required
# # load_required(force_reinstall_pytorch=True)

In [140]:
import torch
torch.cuda.empty_cache()

In [141]:
import os
print(os.getcwd())

d:\Cornell\course\CS6386\Analyzing-the-Correlation-Between-Retail-Traders--Sentiments-and-Equity-Market-Movements\Sentiment_Analysis


In [142]:
# !pip install datasets

https://huggingface.co/nickwong64/bert-base-uncased-finance-sentiment?text=Muted+Fed%2C+AI+hopes+send+Wall+Street+higher

In [143]:
# # # Hugging Face pre-trained
# # Load model directly
# tokenizer = AutoTokenizer.from_pretrained("nickwong64/bert-base-uncased-finance-sentiment")
# model = AutoModelForSequenceClassification.from_pretrained("nickwong64/bert-base-uncased-finance-sentiment")

https://huggingface.co/ProsusAI/finbert?text=Stocks+rallied+and+the+British+pound+gained.

In [144]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [145]:
# # checkpoint = "bert-base-uncased"
# # checkpoint = "bert-base-cased"
# checkpoint = "distilbert-base-uncased"
# # model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# Importing necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Loading the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Function to classify sentiment of multiple sentences
def classify_sentiments(sentences):
    results = []

    for sentence in sentences:
        # Tokenizing the input sentence
        inputs = tokenizer(sentence, return_tensors="pt")

        # Getting model predictions
        with torch.no_grad():
            outputs = model(**inputs)

        # Processing the output logits
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        sentiment = ['negative', 'neutral', 'positive'][probabilities.argmax().item()]

        results.append((sentence, sentiment, probabilities[0].tolist()))

    return results

# List of sentences
sentences = [
    "I had a great experience with this investment.",
    "The market is going to crash soon.",
    "This company's financial outlook is uncertain."
]

# Classify the sentiments
results = classify_sentiments(sentences)

# Print the results
for sentence, sentiment, probabilities in results:
    print(f"Sentence: {sentence}")
    print(f"Sentiment: {sentiment}")
    print(f"Probabilities: {probabilities}\n")


In [146]:
raw_datasets = load_from_disk("data/finetune_data")

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

Map: 100%|██████████| 80793/80793 [00:10<00:00, 7971.72 examples/s]


In [147]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8925
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2232
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1300
    })
    test_unlabeled: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80793
    })
})

In [148]:
# def predict_labels_and_create_dataframe(input_text):
#     # Tokenize the input text
#     inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")
    
#     # Make predictions using the model
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits
    
#     # Convert logits to probabilities using softmax
#     probabilities = torch.softmax(logits, dim=1)
    
#     # Get the predicted class labels (argmax of probabilities)
#     predicted_labels = torch.argmax(probabilities, dim=1).cpu().numpy()
    
#     # Extract the probabilities as Python lists
#     probabilities_list = probabilities.tolist()
    
#     return predicted_labels, probabilities_list

# # Create empty lists to store data
# all_texts = []  # Original text
# all_predicted_labels = []  # Predicted labels
# all_probabilities = []  # Probabilities

# # Iterate through the test_unlabeled dataset
# for text in tokenized_datasets['train']['text']:
#     predicted_labels, probabilities = predict_labels_and_create_dataframe(text)
#     all_texts.append(text)
#     all_predicted_labels.append(predicted_labels[0])  # Extract the scalar label from the NumPy array
#     all_probabilities.append(probabilities[0])  # Extract the inner list

# # Create a DataFrame
# data = {
#     'Text': all_texts,
#     'Predicted_Labels': all_predicted_labels,
#     'Probability_Class_0': [prob[0] for prob in all_probabilities],
#     'Probability_Class_1': [prob[1] for prob in all_probabilities],
#     'Probability_Class_2': [prob[2] for prob in all_probabilities],  # Assuming 3 classes
# }


# df = pd.DataFrame(data)
# output_csv_path = "data/predictions_3labels.csv"
# df.to_csv(output_csv_path, index=False)
# print(f"Predictions have been saved to {output_csv_path}")

In [149]:
import pandas as pd

def predict_labels_and_create_dataframe(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")
    
    # Make predictions using the model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Convert logits to probabilities using softmax
    probabilities = torch.softmax(logits, dim=1)
    
    # Get the predicted class labels (argmax of probabilities)
    predicted_labels = torch.argmax(probabilities, dim=1).cpu().numpy()
    
    # Extract the probabilities as Python lists
    probabilities_list = probabilities.tolist()
    
    return predicted_labels, probabilities_list

# Create empty lists to store data
all_texts = []  # Original text
all_predicted_labels = []  # Predicted labels
all_probabilities = []  # Probabilities

# Iterate through the test_unlabeled dataset
for text in tokenized_datasets['train']['text']:
    predicted_labels, probabilities = predict_labels_and_create_dataframe(text)
    all_texts.append(text)
    # Assign labels based on which probability is larger
    predicted_label = 0 if probabilities[0][0] > probabilities[0][1] else 1
    all_predicted_labels.append(predicted_label)
    all_probabilities.append(probabilities[0][:2])  # Extract the first two probabilities

# Create a DataFrame
data = {
    'Text': all_texts,
    'Predicted_Labels': all_predicted_labels,
    'Probability_Class_0': [prob[0] for prob in all_probabilities],
    'Probability_Class_1': [prob[1] for prob in all_probabilities],  # Assuming 2 classes
}

df = pd.DataFrame(data)
output_csv_path = "data/predictions_2labels.csv"  # Since you have two classes now
df.to_csv(output_csv_path, index=False)
print(f"Predictions have been saved to {output_csv_path}")


Predictions have been saved to data/predictions_2labels.csv


In [150]:
df

,Text,Predicted_Labels,Probability_Class_0,Probability_Class_1
0,$mcp rolling over for a new intraday low. Bott...,1,0.141893,0.723007
1,Lloyds to cut 945 jobs as part of three-year r...,1,0.008439,0.970744
2,The most loyal customers were found in the Ban...,0,0.679721,0.011259
3,Two trades I'm eying GOOG and GD.,0,0.066658,0.025634
4,aapl daily (see notes),0,0.045080,0.038785
...,...,...,...,...
8920,eading stocks analysis updated Failing breako...,1,0.037187,0.607914
8921,Seat belt recall? who cares.... I'm a buyer $...,1,0.055408,0.070968
8922,$SE wave 4 complete post earnings dump show be...,1,0.062176,0.086359
8923,Handelsbanken ranked before Local Cooperative ...,0,0.927398,0.014981


In [152]:
true_labels = tokenized_datasets['train']['label']

# Calculate accuracy
accuracy = accuracy_score(true_labels, df['Predicted_Labels'])

# Calculate precision, recall, and F1 score for each class
precision = precision_score(true_labels, df['Predicted_Labels'], average=None)
recall = recall_score(true_labels, df['Predicted_Labels'], average=None)
f1 = f1_score(true_labels, df['Predicted_Labels'], average=None)

# Calculate macro-average precision, recall, and F1 score
macro_precision = precision_score(true_labels, df['Predicted_Labels'], average='macro')
macro_recall = recall_score(true_labels, df['Predicted_Labels'], average='macro')
macro_f1 = f1_score(true_labels, df['Predicted_Labels'], average='macro')

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Macro-Average Precision: {macro_precision}")
print(f"Macro-Average Recall: {macro_recall}")
print(f"Macro-Average F1 Score: {macro_f1}")

Accuracy: 0.19988795518207284
Precision: [0.16844007 0.23960446]
Recall: [0.21860344 0.18576764]
F1 Score: [0.19027101 0.20927915]
Macro-Average Precision: 0.20402226737464435
Macro-Average Recall: 0.20218554115144788
Macro-Average F1 Score: 0.19977507768940256
